# First, Peak, and Last Years
For each element type and each type of station in North America, find the earliest year, peak year for number of stations, and last year.

In [1]:
# import the dependencies
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
CSV_PATH = os.path.join("static", "data", "csv")
IMG_PATH = os.path.join("static", "img")
FILE_NAME = "full_inventory.csv"
FILE_PATH = os.path.join(CSV_PATH, FILE_NAME)

In [3]:
# Read in the inventory file
# because 'NA' is the code for North America, we need to change pandas.read_csv default behavior
# for reading missing values (na's)...

df = pd.read_csv(FILE_PATH, keep_default_na=False, na_values=['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan',
    '1.#IND', '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan',
    'null'])

C:\Users\dadra\anaconda\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10,11,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Examine the Stations in North America

In [4]:
na_df = df.query("continent == 'North America'")
print(na_df.shape)
na_df.head()

(571534, 19)


,station_id,latitude,longitude,element,first_year,last_year,years,elevation,state,name,gsn_flag,hcn_crn_flag,wmo_id,state_name,country_code,country,continent_code,continent,type
0,ACW00011604,17.1167,-61.7833,TMAX,1949,1949,1,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN,NaN,AC,Antigua and Barbuda,NA,North America,W
1,ACW00011604,17.1167,-61.7833,TMIN,1949,1949,1,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN,NaN,AC,Antigua and Barbuda,NA,North America,W
2,ACW00011604,17.1167,-61.7833,PRCP,1949,1949,1,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN,NaN,AC,Antigua and Barbuda,NA,North America,W
3,ACW00011604,17.1167,-61.7833,SNOW,1949,1949,1,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN,NaN,AC,Antigua and Barbuda,NA,North America,W
4,ACW00011604,17.1167,-61.7833,SNWD,1949,1949,1,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN,NaN,AC,Antigua and Barbuda,NA,North America,W


In [5]:
na_prcp = na_df.query('element == "PRCP"')

In [6]:
na_prcp.shape  # Number of stations in North America reporting PRCP (at some point in time)

(74440, 19)

In [7]:
na_prcp.drop_duplicates('station_id').shape  # sanity check!  no duplicate stations after filtering on elements

(74440, 19)

# Consider the Different Types of Stations in North America

In [8]:
df['ctype'] = df.type.map(lambda x: str(x)) # type type column had issues with 1 and '1' being different - fix here
na_df = df.query("continent == 'North America'")
na_stations = na_df.drop_duplicates('station_id')
na_stations.ctype.value_counts()

1    35986
C    22667
0     7931
N     5170
W     1794
R     1509
S      859
M      107
E       47
P        3
Name: ctype, dtype: int64

In [9]:
def count_station_types_by_year(inventory, element):
    
    def initialize_dict_from_array(arr):
        return {item : 0 for item in arr}
    
    # range of years
    min_year = inventory[inventory['element'] == element].first_year.min()
    max_year = inventory[inventory['element'] == element].last_year.max()
    sub_inventory = inventory[inventory['element'] == element]
    
    types = inventory.station_id.apply(lambda x: x[2]).unique()
    
    results_list = []
    # Not the most efficient - but need to figure out why the above failed...
    for year in range(min_year, max_year + 1):
        results_dict = initialize_dict_from_array(types)
        results_dict['year'] = year
        
        for row in sub_inventory.itertuples():
            if ((row.first_year <= year) and (year <= row.last_year)):
                # station was active in the given year:
                station = row.station_id
                
                # get the station type:
                station_type = row.station_id[2]
                results_dict[station_type] += 1
                
        results_list.append(results_dict)
        
    return pd.DataFrame(results_list)

In [10]:
na_prcp_by_type = count_station_types_by_year(na_df, 'PRCP')
na_prcp_by_type.tail()

,0,1,C,E,M,N,P,R,S,W,year
179,1291,19259,6968,3,94,0,0,0,853,1165,2015
180,1227,19623,6816,3,94,0,0,0,850,1164,2016
181,1168,19798,6697,3,89,0,0,0,832,1159,2017
182,1125,19748,6498,3,89,0,0,0,770,1155,2018
183,1066,18134,6219,0,88,0,0,0,764,1150,2019


In [11]:
na_prcp_by_type.head()

,0,1,C,E,M,N,P,R,S,W,year
0,0,0,0,0,0,0,0,0,0,1,1836
1,0,0,0,0,0,0,0,0,0,1,1837
2,0,0,0,0,0,0,0,0,0,1,1838
3,0,0,0,0,0,0,0,0,0,1,1839
4,1,0,1,0,0,0,0,0,0,1,1840


In [12]:
# The following are helper functions:

def nonzero_indices(ser):
    """Return the nonzero indices of a pandas series."""
    return [i for i, element in enumerate(ser) if element != 0]

def first_nonzero_index(ser):
    """Returns the first nonzero index of a pandas series."""
    nonzero = nonzero_indices(ser)
    if nonzero:
        return min(nonzero)
    else:
        return None
    
def last_nonzero_index(ser):
    """Returns the last nonzero index of a pandas series."""
    nonzero = nonzero_indices(ser)
    if nonzero:
        return max(nonzero)
    else:
        return None

def max_nonzero_index(ser):
    """Returns the (first) index of the largest, nonzero element of a pandas series."""
    nonzero = nonzero_indices(ser)
    if nonzero:
        return ser.idxmax()
    else:
        return None

In [13]:
# Code to find the last nonzero index of each column in a pandas DataFrame
def last_nonzero(df):
    """
    Returns the position of the last nonzero element in each column of the DataFrame df.
    """
    results = {}
    
    for column in df.columns:
        results[column] = last_nonzero_index(df[column])

    return results

def first_nonzero(df):
    """
    Returns the position of the first nonzero element in each column of the DataFrame df.
    """
    results = {}
    
    for column in df.columns:
        results[column] = first_nonzero_index(df[column])
        
    return results

def max_nonzero(df):
    """
    Returns the position of the largest nonzero element in each column of the DataFrame df.
    """
    results = {}
    
    for column in df.columns:
        results[column] = max_nonzero_index(df[column])
        
    return results

In [14]:
last_nonzero(na_prcp_by_type)

{'0': 183,
 '1': 183,
 'C': 183,
 'E': 182,
 'M': 183,
 'N': 178,
 'P': 56,
 'R': None,
 'S': 183,
 'W': 183,
 'year': 183}

In [15]:
first_nonzero(na_prcp_by_type)

{'0': 4,
 '1': 162,
 'C': 4,
 'E': 37,
 'M': 102,
 'N': 66,
 'P': 20,
 'R': None,
 'S': 142,
 'W': 0,
 'year': 0}

In [16]:
max_nonzero(na_prcp_by_type)

{'0': 138,
 '1': 181,
 'C': 115,
 'E': 130,
 'M': 160,
 'N': 147,
 'P': 27,
 'R': None,
 'S': 178,
 'W': 176,
 'year': 183}

In [17]:
def first_peak_last_years(df):
    """
    Return the first nonzero year, largest year, and last nonzero year of each column of a dataframe
    """
    def lookup_if_exists(ser, index):
        if index or (index == 0):
            return ser[index]
        else:
            return np.nan
        
    # first_nonzero_index = df.ne(0).idxmax().to_dict()
    first_nonzero_index = first_nonzero(df)
    #max_index = df.idxmax().to_dict()
    max_index = max_nonzero(df)
    last_nonzero_index = last_nonzero(df)
    
    results = pd.DataFrame()
    results['station_type'] = first_nonzero_index.keys()
    results['first_year'] = np.nan
    results['last_year'] = np.nan
    results['peak_year'] = np.nan
    results.set_index('station_type', inplace=True)
    
    for key in first_nonzero_index.keys():
        results.loc[key, 'first_year'] = lookup_if_exists(df.year, first_nonzero_index[key])
        results.loc[key, 'peak_year'] = lookup_if_exists(df.year, max_index[key])
        results.loc[key, 'last_year'] = lookup_if_exists(df.year, last_nonzero_index[key])
        
    return results


In [18]:
first_peak_last_years(na_prcp_by_type)

,first_year,last_year,peak_year
station_type,,,
0,1840.0,2019.0,1974.0
1,1998.0,2019.0,2017.0
C,1840.0,2019.0,1951.0
E,1873.0,2018.0,1966.0
M,1938.0,2019.0,1996.0
N,1902.0,2014.0,1983.0
P,1856.0,1892.0,1863.0
R,NaN,NaN,NaN
S,1978.0,2019.0,2014.0


In [19]:
na_tmin = count_station_types_by_year(na_df, "TMIN")
na_tmin.tail()

,0,1,C,E,M,N,P,R,S,W,year
175,1287,0,5231,17,96,0,0,1292,853,1156,2015
176,1237,0,5107,17,95,0,0,1281,849,1156,2016
177,1185,0,5011,17,90,0,0,1272,832,1152,2017
178,1145,0,4873,17,90,0,0,1259,770,1149,2018
179,1091,0,4706,2,88,0,0,1234,764,1144,2019


In [20]:
na_tmin.head()

,0,1,C,E,M,N,P,R,S,W,year
0,1,0,0,0,0,0,0,0,0,0,1840
1,1,0,0,0,0,0,0,0,0,0,1841
2,1,0,0,0,0,0,0,0,0,0,1842
3,1,0,0,0,0,0,0,0,0,0,1843
4,1,0,0,0,0,0,0,0,0,0,1844


In [21]:
first_peak_last_years(na_tmin)

,first_year,last_year,peak_year
station_type,,,
0,1840.0,2019.0,1974.0
1,NaN,NaN,NaN
C,1853.0,2019.0,1970.0
E,1873.0,2019.0,1962.0
M,1938.0,2019.0,1996.0
N,1902.0,2014.0,1983.0
P,1877.0,1892.0,1877.0
R,1983.0,2019.0,2005.0
S,1979.0,2019.0,2014.0


In [22]:
na_tmax = count_station_types_by_year(na_df, "TMAX")
na_tmax.head()

,0,1,C,E,M,N,P,R,S,W,year
0,1,0,0,0,0,0,0,0,0,0,1840
1,1,0,0,0,0,0,0,0,0,0,1841
2,1,0,0,0,0,0,0,0,0,0,1842
3,1,0,0,0,0,0,0,0,0,0,1843
4,1,0,0,0,0,0,0,0,0,0,1844


In [23]:
na_tmax.tail()

,0,1,C,E,M,N,P,R,S,W,year
175,1288,0,5231,17,96,0,0,1292,853,1156,2015
176,1236,0,5108,17,95,0,0,1281,849,1156,2016
177,1186,0,5012,17,90,0,0,1272,832,1152,2017
178,1145,0,4875,17,90,0,0,1259,769,1149,2018
179,1089,0,4708,2,88,0,0,1234,763,1144,2019


In [24]:
first_peak_last_years(na_tmax)

,first_year,last_year,peak_year
station_type,,,
0,1840.0,2019.0,1974.0
1,NaN,NaN,NaN
C,1853.0,2019.0,1970.0
E,1876.0,2019.0,1962.0
M,1938.0,2019.0,1996.0
N,1916.0,2014.0,1983.0
P,1877.0,1892.0,1877.0
R,1983.0,2019.0,2005.0
S,1978.0,2019.0,2014.0


In [25]:
na_snow = count_station_types_by_year(na_df, "SNOW")
na_snow.head()

,0,1,C,E,M,N,P,R,S,W,year
0,1,0,0,0,0,0,0,0,0,0,1840
1,1,0,0,0,0,0,0,0,0,0,1841
2,1,0,0,0,0,0,0,0,0,0,1842
3,1,0,0,0,0,0,0,0,0,0,1843
4,1,0,0,0,0,0,0,0,0,0,1844


In [26]:
na_snow.tail()

,0,1,C,E,M,N,P,R,S,W,year
175,604,17914,5957,0,0,0,0,0,0,701,2015
176,539,17800,5817,0,0,0,0,0,0,703,2016
177,478,17695,5660,0,0,0,0,0,0,708,2017
178,438,17102,5438,0,0,0,0,0,0,708,2018
179,398,14749,5076,0,0,0,0,0,0,657,2019


In [27]:
first_peak_last_years(na_snow)

,first_year,last_year,peak_year
station_type,,,
0,1840.0,2019.0,1974.0
1,1998.0,2019.0,2014.0
C,1846.0,2019.0,1950.0
E,NaN,NaN,NaN
M,1949.0,1995.0,1949.0
N,NaN,NaN,NaN
P,1862.0,1892.0,1862.0
R,NaN,NaN,NaN
S,NaN,NaN,NaN


In [28]:
na_snwd = count_station_types_by_year(na_df, "SNWD")
na_snwd.head()

,0,1,C,E,M,N,P,R,S,W,year
0,0,0,1,0,0,0,0,0,0,0,1857
1,0,0,1,0,0,0,0,0,0,0,1858
2,0,0,1,0,0,0,0,0,0,0,1859
3,0,0,1,0,0,0,0,0,0,0,1860
4,0,0,1,0,0,0,0,0,0,0,1861


In [29]:
na_snwd.tail()

,0,1,C,E,M,N,P,R,S,W,year
158,994,9388,5938,0,6,0,0,0,850,730,2015
159,937,9194,5797,0,6,0,0,0,849,728,2016
160,870,9519,5646,0,6,0,0,0,834,730,2017
161,825,9239,5405,0,2,0,0,0,773,719,2018
162,757,6981,5041,0,1,0,0,0,768,661,2019


In [30]:
first_peak_last_years(na_snwd)

,first_year,last_year,peak_year
station_type,,,
0,1933.0,2019.0,1991.0
1,1998.0,2019.0,2013.0
C,1857.0,2019.0,1950.0
E,1958.0,2008.0,1964.0
M,1949.0,2019.0,1993.0
N,NaN,NaN,NaN
P,NaN,NaN,NaN
R,NaN,NaN,NaN
S,1978.0,2019.0,2015.0


In [31]:
# Where are the type 'P' stations coming from?
na_df.drop_duplicates('station_id').query('ctype == "P"').country.value_counts(normalize=True)

United States    1.0
Name: country, dtype: float64

# Maximum Number of Active Stations in a Given Year
By element type and type

In [33]:
na_prcp_by_type.max(axis=0)

0        2998
1       19798
C       10999
E          30
M         103
N        4093
P           2
R           0
S         853
W        1254
year     2019
dtype: int64

In [34]:
na_tmin.max()

0       2429
1          0
C       6365
E         17
M        103
N       3956
P          2
R       1384
S        854
W       1248
year    2019
dtype: int64

In [35]:
na_tmax.max()

0       2429
1          0
C       6380
E         17
M        103
N       3957
P          2
R       1384
S        854
W       1248
year    2019
dtype: int64

In [36]:
na_snow.max()

0        2959
1       17915
C        9790
E           0
M           1
N           0
P           1
R           0
S           0
W         856
year     2019
dtype: int64

In [37]:
na_snwd.max()

0       2603
1       9831
C       9710
E         26
M         16
N          0
P          0
R          0
S        850
W        967
year    2019
dtype: int64